In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:
# Load the training data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Basic info
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Preview the data
train_df.head()

In [ ]:
train_df['rule_violation'].value_counts(normalize=True)

In [ ]:
print("Unique training rules:", train_df['rule'].nunique())
print("Unique test rules:", test_df['rule'].nunique())

In [ ]:
train_rules = set(train_df['rule'].unique())
test_rules = set(test_df['rule'].unique())

print("Unseen test rules:", len(test_rules - train_rules))

In [ ]:
train_df['body_len'] = train_df['body'].apply(lambda x: len(str(x).split()))
train_df['body_len'].describe()

In [ ]:
rule_violation_by_rule = train_df.groupby('rule')['rule_violation'].mean().sort_values()
rule_violation_by_rule.plot(kind='barh', figsize=(10, 10), title='Violation rate per rule')

In [ ]:
train_df.head()

In [ ]:
train_df.describe()


In [ ]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
import nltk

# Tải lexicon nếu chưa có
nltk.download('opinion_lexicon')
nltk.download('punkt')

# Giả sử df có cột 'body' (bình luận) và 'created_utc' (timestamp)


In [ ]:
train_df['Length_char'] = train_df['body'].str.len()   # số ký tự
train_df['Length_word'] = train_df['body'].str.split().str.len()   # số từ


In [ ]:
negative_words = set(opinion_lexicon.negative())

def count_neg_words(text):
    words = word_tokenize(text.lower())
    return sum(1 for w in words if w in negative_words)

train_df['NegWords'] = train_df['body'].apply(count_neg_words)


In [ ]:
def uppercase_ratio(text):
    if len(text) == 0:
        return 0
    num_upper = sum(1 for c in text if c.isupper())
    return num_upper / len(text)

train_df['UpperCaseRatio'] = train_df['body'].apply(uppercase_ratio)


In [ ]:
train_df['ContainsLink'] = train_df['body'].str.contains(r'http[s]?://|www\.', regex=True).astype(int)


In [ ]:
!export HF_ENDPOINT=https://hf-mirror.com



In [2]:
!pip install -q diffusers transformers accelerate peft



^C


In [1]:
from transformers import pipeline
import os
# os.environ["TRANSFORMERS_OFFLINE"] = "1"

sentiment_model = pipeline("sentiment-analysis")


2025-09-28 17:28:52.785029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759080532.981375      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759080533.039302      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Ép buộc model chạy CPU
device = torch.device("cpu")
model.to(device)

# Tạo pipeline nhưng gắn model + tokenizer thủ công
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=-1)


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
def get_sentiment_score(text):
    result = sentiment_model(text)[0]
    label = result['label']   # LABEL_0 (negative), LABEL_1 (neutral), LABEL_2 (positive)
    score = result['score']   # confidence
    if label == "LABEL_0":
        return -score
    elif label == "LABEL_2":
        return score
    else:
        return 0
train_df['SentimentScore'] = train_df['body'].apply(get_sentiment_score)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine body + rule text as input
train_df['text'] = train_df['body'] + " [SEP] " + train_df['rule']

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(train_df['text'], train_df['rule_violation'], test_size=0.2, random_state=42)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=100000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)
val_preds = lr.predict_proba(X_val_tfidf)[:, 1]
print("Logistic Regression AUC:", roc_auc_score(y_val, val_preds))

In [ ]:
# from sklearn.linear_model import Ridge
# from sklearn.metrics import roc_auc_score

# ridge = Ridge(alpha=1.0)
# ridge.fit(X_train_tfidf, y_train)
# val_preds_ridge = ridge.predict(X_val_tfidf)

# print("Ridge Regression AUC:", roc_auc_score(y_val, val_preds_ridge))

In [ ]:
import lightgbm as lgb

# Define datasets
lgb_train = lgb.Dataset(X_train_tfidf, label=y_train)
lgb_val = lgb.Dataset(X_val_tfidf, label=y_val)

# Params
params = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    'boosting_type': 'gbdt'
}

# Use callbacks for early stopping
callbacks = [lgb.early_stopping(stopping_rounds=10)]

# Train
lgb_model = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=[lgb_val],
    valid_names=["val"],
    callbacks=callbacks
)

In [ ]:
val_preds_lgb = lgb_model.predict(X_val_tfidf, num_iteration=lgb_model.best_iteration)
from sklearn.metrics import roc_auc_score
print("LightGBM AUC:", roc_auc_score(y_val, val_preds_lgb))

In [ ]:
# Combine body and rule just like training
test_df['input'] = test_df['body'] + ' [SEP] ' + test_df['rule']
X_test_tfidf = vectorizer.transform(test_df['input'])

# Predict
test_preds = lgb_model.predict(X_test_tfidf, num_iteration=lgb_model.best_iteration)

# Prepare submission
submission = pd.DataFrame({
    'row_id': test_df['row_id'],
    'rule_violation': test_preds
})

submission.to_csv('submission.csv', index=False)